In [3]:
!pip install keras-tuner
!pip install tensorflow


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
#Drop additional columms to see if that inproves model accuracy
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS', 'AFFILIATION'], axis=1)
application_df.head()

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,1,0,5000,1
1,T3,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,C3000,ProductDev,Association,1,0,5000,0
3,T3,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [6]:
# Determine the number of unique values in each column.

application_cat = application_df.columns.tolist()
application_df[application_cat].nunique()


APPLICATION_TYPE      17
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
app_type_value_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_value_counts[app_type_value_counts > 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

Other    34023
T9         156
T13         66
T12         27
T2          16
T25          3
T14          3
T29          2
T15          2
T17          1
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
call_type_value_counts = application_df['CLASSIFICATION'].value_counts()
call_type_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
call_type_value_counts[call_type_value_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(call_type_value_counts[call_type_value_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
#what are the different column types
application_df.dtypes

APPLICATION_TYPE    object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
STATUS               int64
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [13]:
#See what the value_counts are for income amounts
income_amt_value_counts = application_df['INCOME_AMT'].value_counts()
income_amt_value_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [14]:
# Choose a cutoff value and create a list of Income classification to be replaced

income_to_replace = list(income_amt_value_counts[income_amt_value_counts < 2000].index)

# Replace in dataframe
for inc in income_to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(inc,"Other")

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [15]:
#put columns that are objects into a list to be categroized with get dummies
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [16]:
OneHotEncoder,
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,APPLICATION_TYPE_T9,...,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [17]:
# merge the onehotencoded features and drop originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

<ipython-input-17-013df3c00c86>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  application_df = application_df.drop(application_cat,1)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T2,...,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,108590,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1,142590,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [18]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(['IS_SUCCESSFUL'], axis='columns').values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
print(len(X_train_scaled[0]))

31


In [21]:
#First, lets try running Keras Tuner again with the new amount of features
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  15
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                480       
                                                                 
 dense_1 (Dense)             (None, 15)                240       
                                                                 
 dense_2 (Dense)             (None, 15)                240       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 976
Trainable params: 976
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 6s 4ms/step - loss: 0.6555 - accuracy: 0.6050
Epoch 2/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6412 - accuracy: 0.6254
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6398 - accuracy: 0.6288
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6385 - accuracy: 0.6310
Epoch 5/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6377 - accuracy: 0.6303
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.6370 - accuracy: 0.6324
Epoch 7/200
804/804 [==============================] - 3s 3ms/step - loss: 0.6369 - accuracy: 0.6312
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.6362 - accuracy: 0.6321
Epoch 9/200
804/804 [==============================] - 4s 5ms/step - loss: 0.6358 - accuracy: 0.6313
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.6355 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6361 - accuracy: 0.6365 - 509ms/epoch - 2ms/step
Loss: 0.6360600590705872, Accuracy: 0.6365014314651489


In [25]:
#Try again but go back to original dataframe drop the same two initial identifying columns, add back in other columns
#Import and read the charity_data.csv.

application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [26]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [27]:
#BIN THE APPLICATION_TYPE AND and add back in
app_type_value_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_value_counts[app_type_value_counts > 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

Other    34023
T9         156
T13         66
T12         27
T2          16
T25          3
T14          3
T29          2
T15          2
T17          1
Name: APPLICATION_TYPE, dtype: int64

In [28]:
#bin CLASSIFICATION and add back to dataframe
call_type_value_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(call_type_value_counts[call_type_value_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [29]:
# Convert categorical data to numeric with `pd.get_dummies`
application_with_dummies_df = pd.get_dummies(application_df)
application_with_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [30]:
X = 0
y = 0
X_train = 0
y_train = 0
X_train_scaled = 0
X_test_scaled = 0

In [31]:
# Split our preprocessed data into our features and target arrays
X = application_with_dummies_df.drop(['IS_SUCCESSFUL'], axis='columns').values
y = application_with_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [32]:
# Create a StandardScaler instances
new_scaler = StandardScaler()

# Fit the StandardScaler
new_scaler.fit(X_train)

# Scale the data
X_train_new_scaled = new_scaler.transform(X_train)
X_test_new_scaled = new_scaler.transform(X_test)

In [33]:
print(len(X_train_new_scaled[0]))

44


In [34]:
#try running with different activation function
# Create a Keras Sequential model and add more than one Dense hidden layer
nn_model_new = tf.keras.models.Sequential()

nn_model_new.add(tf.keras.layers.Dense(units=15, activation="tanh", input_dim=44))

nn_model_new.add(tf.keras.layers.Dense(units=15, activation="tanh"))

nn_model_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model_new.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 15)                675       
                                                                 
 dense_5 (Dense)             (None, 15)                240       
                                                                 
 dense_6 (Dense)             (None, 1)                 16        
                                                                 
Total params: 931
Trainable params: 931
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model and train over more than 200 epochs
nn_model_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model_new = nn_model_new.fit(X_train_new_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.6128 - accuracy: 0.6806
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5898 - accuracy: 0.6993
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5825 - accuracy: 0.7045
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5794 - accuracy: 0.7075
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5777 - accuracy: 0.7086
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5767 - accuracy: 0.7105
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5757 - accuracy: 0.7101
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5751 - accuracy: 0.7099
Epoch 9/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5749 - accuracy: 0.7105
Epoch 10/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5743 - accura

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_new.evaluate(X_test_new_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5816 - accuracy: 0.7037 - 463ms/epoch - 2ms/step
Loss: 0.5816230773925781, Accuracy: 0.7036734819412231


In [37]:
#try running with relu activation function but this time with 30 nodes and another layer
# Create a Keras Sequential model and add more than one Dense hidden layer
nn_model_relu = tf.keras.models.Sequential()

nn_model_relu.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=44))

nn_model_relu.add(tf.keras.layers.Dense(units=30, activation="relu"))

nn_model_relu.add(tf.keras.layers.Dense(units=30, activation="relu"))

nn_model_relu.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model_relu.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 30)                1350      
                                                                 
 dense_8 (Dense)             (None, 30)                930       
                                                                 
 dense_9 (Dense)             (None, 30)                930       
                                                                 
 dense_10 (Dense)            (None, 1)                 31        
                                                                 
Total params: 3,241
Trainable params: 3,241
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn_model_relu.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model_relu = nn_model_relu.fit(X_train_new_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.6031 - accuracy: 0.6894
Epoch 2/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5808 - accuracy: 0.7075
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5777 - accuracy: 0.7093
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5758 - accuracy: 0.7095
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5756 - accuracy: 0.7112
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5751 - accuracy: 0.7112
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5741 - accuracy: 0.7133
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5735 - accuracy: 0.7130
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5736 - accuracy: 0.7125
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5729 - accura

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_relu.evaluate(X_test_new_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5966 - accuracy: 0.7054 - 475ms/epoch - 2ms/step
Loss: 0.596604585647583, Accuracy: 0.7054227590560913


In [43]:
# Export our model to HDF5 file
from google.colab import drive

drive.mount('/content/drive2')
nn_model_relu.save('/content/drive2/MyDrive/AlphabetSoupCharity_Optimization.h5')

Mounted at /content/drive2
